<a href="https://colab.research.google.com/github/LucasDatilioCarderelli/Maratona_BehindTheCode_IBM21/blob/main/desafio-3/MBTC2021_df3_GFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTC2021-df3_GFT

# Part 1

## Imports

In [ ]:
# !pip install apyori
!pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
answer = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-3-2021/main/assets/data/ANSWERS.csv")
insurance = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-3-2021/main/assets/data/InsuranceCompanyABC.csv")
investment = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-3-2021/main/assets/data/InvestmentBankCDE.csv")
retail = pd.read_csv("https://raw.githubusercontent.com/maratonadev/desafio-3-2021/main/assets/data/RetailBankEFG.csv")
answer.shape, insurance.shape, investment.shape, retail.shape,

((13730, 31), (10082, 9), (10082, 8), (10082, 10))

In [ ]:
answer.head()

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
insurance.head(3)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,71504.0,42.993,4428.512,NE,1.0,T,T,T,T
1,59751.0,26.373,4489.930,CO,0.0,T,T,T,T
2,11877.0,46.977,4567.741,CO,0.0,F,F,F,F


In [ ]:
investment.head(3)

,ID,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto
0,71504.0,F,F,F,F,F,F,F
1,59751.0,F,F,F,F,F,F,F
2,11877.0,F,F,F,F,F,F,F


In [ ]:
retail.head(3)

,ID,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.0,T,T,T,T,T,F,F,T,F
1,59751.0,T,T,T,T,T,F,F,F,T
2,11877.0,F,F,F,F,F,F,F,F,F


## Preprocessing

In [ ]:
def clean_df(df):
    object_columns = df.select_dtypes("object")
    for column in object_columns:
        df[column].replace({"T" : np.True_, "F" : np.False_}, inplace=True)
    return df

In [ ]:
df = insurance.merge(investment, on="ID").merge(retail, on="ID", sort=True)
# df = df.drop_duplicates().set_index("ID")
# df.index = df.index.astype(int).astype(str)
# clean_df(df)
# df = df.iloc[:,4:]
df

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F
13726,100962.0,20.505,6809.895,SE,0.0,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F
13727,100963.0,21.067,6201.485,SE,1.0,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F
13728,100978.0,43.481,8130.084,N,0.0,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F


In [ ]:
(answer.ID == df.ID).sum()

13730

## Apriori

In [ ]:
frequent_itemsets = apriori(df, min_support=0.10, use_colnames=True, max_len=5)
frequent_itemsets

,support,itemsets
0,0.278951,(seguro auto)
1,0.342535,(seguro vida Emp)
2,0.250182,(seguro vida PF)
3,0.332484,(Seguro Residencial)
4,0.345448,(Investimento Fundos_cambiais)
...,...,...
152,0.124545,"(Investimento Fundos_commodities, Investimento..."
153,0.106701,"(Investimento Fundos_commodities, Investimento..."
154,0.115222,"(Emprestimo _viagem, Investimento CDB, Emprest..."
155,0.121413,"(Investimento LCI, Investimento Poupanca, Inve..."


In [ ]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.80)
rules.sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
36,"(Emprestimo _pessoal, Emprestimo _consignado)",(Emprestimo _limite_especial),0.122870,0.161326,0.119519,0.972733,6.029625,0.099697,30.757473
34,"(Emprestimo _limite_especial, Emprestimo _pess...",(Emprestimo _consignado),0.125346,0.160743,0.119519,0.953515,5.931929,0.099371,18.054518
94,"(Investimento CDB, Emprestimo _educacao)","(Emprestimo _viagem, Investimento Fundos)",0.133503,0.147633,0.115222,0.863066,5.846027,0.095513,6.224657
90,"(Emprestimo _viagem, Investimento CDB, Investi...",(Emprestimo _educacao),0.129862,0.159942,0.115222,0.887269,5.547449,0.094452,7.451860
103,"(Investimento LCI, Investimento Fundos_cambiai...","(Investimento Fundos_commodities, Investimento...",0.144938,0.157902,0.121413,0.837688,5.305103,0.098527,5.188155
...,...,...,...,...,...,...,...,...,...
44,"(Emprestimo _viagem, Investimento Fundos)",(Investimento CDB),0.147633,0.494683,0.129862,0.879625,1.778158,0.056830,4.197857
73,"(Investimento Fundos_commodities, Investimento...",(Investimento CDB),0.121340,0.494683,0.106191,0.875150,1.769112,0.046166,4.047393
87,"(Investimento Fundos_commodities, Investimento...",(Investimento CDB),0.121923,0.494683,0.106701,0.875149,1.769111,0.046387,4.047370
3,(Emprestimo _educacao),(Investimento CDB),0.159942,0.494683,0.133503,0.834699,1.687342,0.054383,3.056958


## Recomendation

In [ ]:
# Disable Warning: A value is trying to be set on a copy of a slice from a DataFrame
pd.options.mode.chained_assignment = None

In [ ]:
answer

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
13726,100962.0,20.505,6809.895,SE,0.0,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F,NaN,NaN,NaN,NaN,NaN,NaN
13727,100963.0,21.067,6201.485,SE,1.0,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
13728,100978.0,43.481,8130.084,N,0.0,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rules[['consequents']].value_counts()

consequents                                                    
(Investimento Fundos_commodities)                                  13
(Investimento Fundos_cambiais)                                     13
(Investimento LCI)                                                 13
(Investimento CDB)                                                 10
(Investimento Poupanca)                                             7
(seguro auto)                                                       6
(Seguro Residencial)                                                6
(seguro vida Emp)                                                   6
(Investimento Fundos)                                               4
(Investimento LCA)                                                  4
(seguro vida PF)                                                    4
(Emprestimo _viagem)                                                3
(Investimento Fundos_commodities, Investimento Fundos_cambiais)     2
(seguro auto, Seguro Resid

In [ ]:
rules['consequents'].unique()

array([frozenset({'seguro auto'}), frozenset({'seguro vida Emp'}),
       frozenset({'Seguro Residencial'}), frozenset({'Investimento CDB'}),
       frozenset({'Investimento Fundos'}), frozenset({'seguro vida PF'}),
       frozenset({'seguro auto', 'Seguro Residencial'}),
       frozenset({'Investimento LCI'}),
       frozenset({'Investimento Fundos_cambiais'}),
       frozenset({'Investimento Fundos_commodities'}),
       frozenset({'Investimento Poupanca'}),
       frozenset({'Emprestimo _consignado'}),
       frozenset({'Emprestimo _pessoal'}),
       frozenset({'Emprestimo _limite_especial'}),
       frozenset({'Emprestimo _viagem'}),
       frozenset({'Emprestimo _educacao'}),
       frozenset({'seguro vida PF', 'seguro vida Emp'}),
       frozenset({'Seguro Residencial', 'seguro vida Emp'}),
       frozenset({'Seguro Residencial', 'seguro vida PF'}),
       frozenset({'seguro auto', 'seguro vida Emp'}),
       frozenset({'seguro auto', 'seguro vida PF'}),
       frozenset({'Inves

In [ ]:
# answer['products'] = np.nan
# for row in answer.index:
#     products = []
#     for col in answer.columns:
#         if answer[col][row] == 'T':
#             products.append(col)
#     answer['products'][row] = products

# count = 0
# for row in range(len(answer)): 
#     if answer['products'][row] in rules['antecedents']:
#         count+=1
# print(count)

In [ ]:
# for row in rules.index:
#     for col in rules['antecedents'][row]:
#         print(col)
#     print(9*'-')

In [ ]:
# # "Emprestimo _limite_especial"
# count = 0
# for row in answer.index:
#     if answer["Emprestimo _consignado"][row] == "T" and \
#        answer["Emprestimo _pessoal"][row] == "T" and \
#        answer["Emprestimo _limite_especial"][row] == "F":
#         answer["RECOMMENDATION_1"][row] = "Emprestimo _limite_especial"
#         count+=1
# print(count)

# Part 2

In [ ]:
flow_output = pd.read_csv("/content/output.csv")
flow_output

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,$A-20 campos-1,$AC-20 campos-1,$A-Rule_ID-1,$A-20 campos-2,$AC-20 campos-2,$A-Rule_ID-2,$A-20 campos-3,$AC-20 campos-3,$A-Rule_ID-3
0,1004.0,40.685,8474.687,CO,1.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,Investimento Fundos,0.900164,12.0,Emprestimo _viagem,0.883252,10.0,NaN,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,Investimento Fundos_cambiais,0.936571,107.0,Investimento LCA,0.847374,86.0,NaN,NaN,NaN
13726,100962.0,20.505,6809.895,SE,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,Emprestimo _pessoal,0.822238,18.0,Emprestimo _limite_especial,0.816572,16.0,NaN,NaN,NaN
13727,100963.0,21.067,6201.485,SE,1.0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13728,100978.0,43.481,8130.084,N,0.0,1,1,1,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,1,0,Investimento Fundos,0.900164,12.0,Emprestimo _viagem,0.883252,10.0,NaN,NaN,NaN


In [ ]:
product_columns = flow_output.iloc[:, 5:-9]
for column in product_columns:
    flow_output[column].replace({1 : "T", 0 : "F"}, inplace=True)
flow_output

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,$A-20 campos-1,$AC-20 campos-1,$A-Rule_ID-1,$A-20 campos-2,$AC-20 campos-2,$A-Rule_ID-2,$A-20 campos-3,$AC-20 campos-3,$A-Rule_ID-3
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,Investimento Fundos,0.900164,12.0,Emprestimo _viagem,0.883252,10.0,NaN,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F,Investimento Fundos_cambiais,0.936571,107.0,Investimento LCA,0.847374,86.0,NaN,NaN,NaN
13726,100962.0,20.505,6809.895,SE,0.0,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F,Emprestimo _pessoal,0.822238,18.0,Emprestimo _limite_especial,0.816572,16.0,NaN,NaN,NaN
13727,100963.0,21.067,6201.485,SE,1.0,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13728,100978.0,43.481,8130.084,N,0.0,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F,Investimento Fundos,0.900164,12.0,Emprestimo _viagem,0.883252,10.0,NaN,NaN,NaN


In [ ]:
flow_output.drop(labels=["$A-Rule_ID-1", 
                         "$A-Rule_ID-2", 
                         "$A-Rule_ID-3"], 
                 axis=1, inplace=True)

In [ ]:
flow_output.rename(columns={
    "$A-20 campos-1":"RECOMMENDATION_1", "$AC-20 campos-1":"CONFIDENCE_1",
    "$A-20 campos-2":"RECOMMENDATION_2", "$AC-20 campos-2":"CONFIDENCE_2",
    "$A-20 campos-3":"RECOMMENDATION_3", "$AC-20 campos-3":"CONFIDENCE_3"
    }, inplace=True)

In [ ]:
flow_output

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,Investimento Fundos,0.900164,Emprestimo _viagem,0.883252,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F,Investimento Fundos_cambiais,0.936571,Investimento LCA,0.847374,NaN,NaN
13726,100962.0,20.505,6809.895,SE,0.0,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F,Emprestimo _pessoal,0.822238,Emprestimo _limite_especial,0.816572,NaN,NaN
13727,100963.0,21.067,6201.485,SE,1.0,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
13728,100978.0,43.481,8130.084,N,0.0,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F,Investimento Fundos,0.900164,Emprestimo _viagem,0.883252,NaN,NaN


In [ ]:
answer

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.0,40.685,8474.687,CO,1.0,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.0,56.693,7996.353,SE,0.0,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
2,1034.0,49.413,8624.335,NE,0.0,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
3,1035.0,22.764,6885.196,S,1.0,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
4,1039.0,22.059,7749.965,NE,1.0,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.0,28.659,8804.506,S,1.0,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN
13726,100962.0,20.505,6809.895,SE,0.0,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F,NaN,NaN,NaN,NaN,NaN,NaN
13727,100963.0,21.067,6201.485,SE,1.0,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
13728,100978.0,43.481,8130.084,N,0.0,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(flow_output["RECOMMENDATION_1"].unique(), 
      flow_output["RECOMMENDATION_1"].count())

['Investimento Fundos' nan 'Investimento LCI'
 'Investimento Fundos_commodities' 'seguro vida PF' 'seguro auto'
 'Investimento CDB' 'Investimento Poupanca' 'Emprestimo _educacao'
 'Seguro Residencial' 'Investimento Fundos_cambiais' 'Emprestimo _viagem'
 'Emprestimo _pessoal' 'Investimento LCA' 'Emprestimo _consignado'
 'Emprestimo _limite_especial' 'seguro vida Emp'] 5841


In [ ]:
flow_output.to_csv("ANSWERS.csv", index=False)